# **Face Mask Detection**

In this notebook I am going to detect the face covering on a person. I have a dataset of 20 different labels like face_with_mask, no_mask, hijab, helmet, sunglasss, hat, etc.

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import tensorflow as tf
import seaborn as sns

In [ ]:
img_path=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images")
anno_path=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/annotations")
train_df = pd.read_csv('../input/face-mask-detection-dataset/train.csv')
submission = pd.read_csv('../input/face-mask-detection-dataset/submission.csv')
print(len(os.listdir(img_path)))
print(len(os.listdir(anno_path)))

In [ ]:
#sorting images based on indexes
images=os.listdir(img_path)
annot=os.listdir(anno_path)
images.sort()
annot.sort()

# **Train/Test Split**

In [ ]:
train_images=images[1698:]
test_images=images[:1698]

In [ ]:
len(train_images)+len(test_images)

Let us visualize a few examples!

In [ ]:
img=plt.imread(os.path.join(img_path,train_images[3]))
plt.imshow(img)
plt.show()

In [ ]:
img=plt.imread(os.path.join(img_path,train_images[10]))
plt.imshow(img)
plt.show()

In [ ]:
train_df.head()

In [ ]:
train_df['name'].nunique()

In [ ]:
#Total classes
train_df['classname'].nunique()

Listing all different categories

In [ ]:
train_df['classname'].value_counts()

In [ ]:
ax = sns.catplot(x='classname',kind='count',data=train_df,orient="h",height=10,aspect=2)
ax.fig.suptitle('Count of Classnames',fontsize=16,color="r")
ax.fig.autofmt_xdate()

This shows a single picture might contain more than one person

In [ ]:
train_df['name'].value_counts().max()

Let us find and plot this image

In [ ]:
train_df.groupby(['name']).count().sort_values(['x1'])

Let us define a function to draw boxes around the faces

In [ ]:
box=[]
for i in range(len(train_df)):
    arr=[]
    for j in train_df.iloc[i][['x1','x2','y1','y2']]:
        arr.append(j)
    box.append(arr)
train_df['box']=box

In [ ]:
def get_boxes(id):
    boxes=[]
    for i in train_df[train_df["name"]==str(id)]["box"]:
        boxes.append(i)
    return boxes

In [ ]:
print(get_boxes(train_images[0]))

In [ ]:
image = '1914.jpg'
img=plt.imread(os.path.join(img_path, image))
fig, ax = plt.subplots(1)
ax.imshow(img)
boxes=get_boxes(image)
for box in boxes:
    rect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],linewidth=2,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
plt.show()

# **Data Preparation**

Cropping the images to the get face images. These facial shots will be used for model training for the classification task

In [ ]:
img_size=50
x=[]
y=[]
for i in range(len(train_df)):
    arr=[]
    for item in train_df.iloc[i]:
        arr.append(item)
    img = cv2.imread(os.path.join(img_path, arr[0]), cv2.IMREAD_GRAYSCALE)
    img = img[arr[2]:arr[4], arr[1]:arr[3]]
    try:
        img = cv2.resize(img, (img_size, img_size))
    except:
        continue
    x.append(img)
    y.append(arr[5])

In [ ]:
with_mask =['face_with_mask']

In [ ]:
#Creating a seperate copy of data
x_mask=[]
y_mask=[]
for i in range(len(x)):
    x_mask.append(x[i])
    y_mask.append(y[i])

## Encoding the labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
lr = LabelEncoder()
y_mask = lr.fit_transform(y_mask)

In [ ]:
#Normalizing the data
x_mask=tf.keras.utils.normalize(x_mask,axis=1)
x_mask=np.array(x_mask).reshape(-1,50,50,1)
from keras.utils import to_categorical
y_mask = to_categorical(y_mask)

In [ ]:
np.shape(x_mask)

# Model Training

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import UnitNorm

from IPython.display import SVG, Image

In [ ]:
X_train,X_val,Y_train,Y_val=train_test_split(x_mask, y_mask,train_size=0.95,random_state=0)

In [ ]:
batch_size = 32
datagen_train = ImageDataGenerator(horizontal_flip=True, rotation_range=20,brightness_range=(0., 2.), shear_range=40)

train_generator = datagen_train.flow(X_mask, Y_mask, batch_size=batch_size, shuffle=True)
#val_generator = datagen_train.flow(X_val, Y_val,batch_size=batch_size, shuffle=True)

In [ ]:
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(50,50,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Convolution layer
model.add(Conv2D(256,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1,1)))

# 5th Convolution layer
model.add(Conv2D(1024,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 6th Convolution layer
model.add(Conv2D(2048,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(20, activation='softmax'))

opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
visualkeras.layered_view(model, to_file='output.png').show()


In [ ]:
epochs = 70
steps_per_epoch = train_generator.n//train_generator.batch_size

callbacks=[ReduceLROnPlateau(monitor='train_loss', patience=2, verbose=1), 
           ]

history = model.fit(train_generator,steps_per_epoch=steps_per_epoch, 
                    epochs=epochs, callbacks=[callbacks])

Train/Val loss plot

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history.history['loss'], label='train')
#axes[0].plot(history.history['val_loss'], label='val')
axes[0].set_title('loss')
axes[0].legend()
axes[1].plot(history.history['accuracy'], label='train')
#axes[1].plot(history.history['val_accuracy'], label='val')
axes[1].set_title('accuracy')
axes[1].legend()

In [ ]:
pip install mtcnn

To detect and localize the facial images from the input images we will be using the MTCNN model

In [ ]:
from mtcnn.mtcnn import MTCNN

In [ ]:
def predict(pic):
    cvNet = cv2.dnn.readNetFromCaffe('../input/caffepretrainedfacemodel/architecture.txt',
                                     '../input/caffepretrainedfacemodel/weights.caffemodel')
    #detector=MTCNN()
    img = plt.imread(pic)
    inp=cv2.imread(pic, cv2.IMREAD_GRAYSCALE)
    (h, w) = inp.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img.copy(), (300,300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    cvNet.setInput(blob)
    detections = cvNet.forward()
    for i in range(0, detections.shape[2]):
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        frame = inp[startY:endY, startX:endX]
        try:
            im = cv2.resize(frame,(50,50))
        except:
            continue
        confidence = detections[0, 0, i, 2]
        if confidence > 0.8:
            cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)
            im = im.reshape(-1,50,50,1)
            pred=model.predict(im)
            pred=np.argmax(pred)
            if pred==14 or pred==15:
                pred=5
            elif pred==3:
                pred=2
            pred=lr.inverse_transform([pred])[0]

    return img,pred

In [ ]:
fig, ax=plt.subplots(1,5,figsize=(20,20))
a=361
for i in range(a,366):
    img,pred=predict(os.path.join(img_path, test_images[i]))
    ax[i-a].imshow(img)
    ax[i-a].set_title(pred)
    ax[i-a].set_yticklabels([])
    ax[i-a].set_xticklabels([])


In [ ]:
d={}
d['Class']=lr.classes_
d['Probability']=a.flatten()
d=pd.DataFrame.from_dict(d)

In [ ]:
sns.barplot(x='Class', y='Probability', data=d)
plt.xticks(rotation=90)

In [ ]:
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(x_mask[23].reshape(1,50,50,1))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            ax[row][col].set_yticklabels([])
            ax[row][col].set_xticklabels([])
            activation_index += 1

In [ ]:
display_activation(activations, 5, 5, 19)

In [ ]:
plt.imshow(x[16])

In [ ]:
len(model.layers)

In [ ]:
fig, ax=plt.subplots(1,20,figsize=(15,10))
for i in range(10,30):
    ax[i-10].imshow(x_mask[i],cmap='gray')

In [ ]:
lr.classes_